In [1]:
import anndata
import numpy as np
import pandas as pd 
import seaborn as sb
import matplotlib.pyplot as plt
import regex as re
import scanpy as sc

import os,sys,inspect

import warnings
warnings.filterwarnings("ignore")

adata_ref = anndata.read_h5ad('..path/to/invivo/with/pseudotime/estimate.h5ad')
adata_query = anndata.read_h5ad('..path/to/invitro/with/pseudotime/estimate.h5ad')

# Genes2Genes runs

In [2]:
# NB to load aligner objects for plotting, saved from below runs at any time:
import pickle

#TF object
with open('aligner_TF.pkl', 'rb') as f:
    aligner = pickle.load(f)
    
#hvg object
with open('aligner_hvgs.pkl', 'rb') as f:
    aligner_hvgs = pickle.load(f)

In [ ]:
#install G2G
pip install git+https://github.com/Teichlab/Genes2Genes.git

In [3]:
#import modules
from genes2genes import Main
from genes2genes import ClusterUtils
from genes2genes import TimeSeriesPreprocessor
from genes2genes import PathwayAnalyser
from genes2genes import VisualUtils

In [12]:
#preprocess
sc.pp.normalize_per_cell(adata_ref, 10000) 
sc.pp.log1p(adata_ref)
sc.pp.normalize_per_cell(adata_query, 10000) 
sc.pp.log1p(adata_query)

In [ ]:
#identify HVGs
sc.pp.highly_variable_genes(adata_ref, subset=False)
sc.pp.highly_variable_genes(adata_query, subset=False)
ref_genes = adata_ref.var_names[adata_ref.var['highly_variable']]
query_genes = adata_query.var_names[adata_query.var['highly_variable']]
hvg_genes = np.intersect1d(ref_genes, query_genes)
len(hvg_genes)

In [ ]:
common_genes = np.intersect1d(adata_query.var_names, adata_ref.var_names)
def get_human_TF_list(): 
    # get a human TF list 
    TF_list = pd.read_csv('/path/to/human/TFlist.csv',skiprows=1)
    np.unique(TF_list['Unnamed: 3'], return_counts=True) # obtain true list without duplicates 
    TF_list = TF_list[TF_list['Unnamed: 3']=='Yes']
    TF_list['Name']
    return TF_list
human_TFs = get_human_TF_list()
human_TFs = np.intersect1d(common_genes , np.asarray(human_TFs['Name']) )
len(human_TFs)

In [ ]:
#see how many genes to test
print(len(hvg_genes))
print(len(human_TFs))

In [20]:
#obtain timeseries
adata_ref.obs['time'] = TimeSeriesPreprocessor.Utils.minmax_normalise(np.asarray(adata_ref.obs['pseudotime']))
adata_query.obs['time'] = TimeSeriesPreprocessor.Utils.minmax_normalise(np.asarray(adata_query.obs['pseudotime']))

In [ ]:
#ensure 0-->1
print(min(adata_ref.obs['time']), max(adata_ref.obs['time']))
print(min(adata_query.obs['time']), max(adata_query.obs['time']))

In [ ]:
# Visualise the two distributions
sb.kdeplot(adata_ref.obs['time'], fill=True, label='Reference - in vivo', color='forestgreen') 
sb.kdeplot(adata_query.obs['time'], fill=True, label='Query - in vitro', color='midnightblue'); 
plt.xlabel('pseudotime'); plt.legend(); plt.show()

In [ ]:
#optimise binning
import optbinning
optbinning.__version__

from optbinning import ContinuousOptimalBinning

x = np.asarray(adata_ref.obs.time)
optb = ContinuousOptimalBinning(name='pseudotime', dtype="numerical")
optb.fit(x, x)
print(len(optb.splits))

x = np.asarray(adata_query.obs.time)
optb = ContinuousOptimalBinning(name='pseudotime', dtype="numerical")
optb.fit(x, x)

#print optimal number of bins
print(len(optb.splits))

In [ ]:
#overlay bins
VisualUtils.plot_pseudotime_dists_with_interpolation_points(adata_ref, adata_query, n_bins)

In [28]:
#visualise bin composition
joint_cmap = {}
colors = np.asarray(sb.color_palette('tab20').as_hex()) 
i=0
for c in np.unique(adata_ref.obs.fineanno):
    joint_cmap[c] = colors[i]
    i+=1
    
for c in np.unique(adata_query.obs.day):
    joint_cmap[c] = colors[i]
    i+=1

In [ ]:
VisualUtils.plot_celltype_barplot(adata_ref, n_bins, 'annotation', joint_cmap=joint_cmap)
VisualUtils.plot_any_legend(joint_cmap)
VisualUtils.plot_celltype_barplot(adata_query, n_bins, 'annotation', joint_cmap=joint_cmap)

In [ ]:
#run alignment for TFs
aligner = Main.RefQueryAligner(adata_ref, adata_query, human_TFs, n_bins) 
aligner.align_all_pairs() 

In [ ]:
#alignment info and visualisation
aligner.get_aggregate_alignment()

In [ ]:
#data frame of this information 
df = aligner.get_stat_df() # ordered genes according to alignment similarity statistics 
df

In [ ]:
#plot any gene of interest
VisualUtils.plotTimeSeries('TGIF1', aligner)

In [ ]:
#write the TF aligner object
import pickle
pickle.dump(aligner, open('aligner_TF.pkl', 'wb')) 

In [ ]:
#run alignment for HVGs
aligner_hvgs = Main.RefQueryAligner(adata_ref, adata_query, hvg_genes, n_bins) #
aligner_hvgs.align_all_pairs() 

In [ ]:
##run alignment for TFs
aligner_hvgs.get_aggregate_alignment()

In [ ]:
#obtain df
df = aligner_hvgs.get_stat_df() # ordered genes according to alignment similarity statistics 
df

In [ ]:
#plot any gene
VisualUtils.plotTimeSeries('EGR1', aligner_hvgs)